In [1]:
import geemap, ee
from GEE_xiong import ee_export
import os

ee.Initialize()
ee_shp = geemap.shp_to_ee(r'C:\Users\xiong\Documents\geemap\shp\qh\qhwgs.shp')
roi = ee_shp.geometry()

In [2]:
def K(image):
    return image.expression(
        'pr/aet',{
            'pr':image.select('pr'),
            'aet':image.select('aet').multiply(0.1)
        }
    ).rename('K')

In [3]:
import numpy as np
precipitation = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")\
                .filterBounds(roi)\
                .filter(ee.Filter.calendarRange(2018,2022,'year'))\
                .select(['pr','aet'])

Ks = precipitation.map(K).mean().float()

In [4]:
Kmax = ee.Number(Ks.reduceRegion(**{
      'reducer': ee.Reducer.max(),
      'geometry': roi,
      'scale': ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE").first().projection().nominalScale().getInfo(),
      'maxPixels': 1e13
    }).get('K'))

Kmax.getInfo()

9.696819305419922

In [5]:
outfile = r'C:\Users\xiong\Downloads\data\K'
ee_export(roi, Ks, outfile, scale = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE").first().projection().nominalScale().getInfo())

Generating URL ...
Please wait ...
Data downloaded to C:\Users\xiong\Downloads\data\K.tif
C:\Users\xiong\Downloads\data\K.tif download successful !!!
